<a href="https://colab.research.google.com/github/Ilona019/ColabAlice/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%B8%D0%B4%D0%B5%D0%BD%D1%82%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8_%D0%B2%D0%B7%D0%BB%D0%BE%D0%BC%D1%89%D0%B8%D0%BA%D0%B0_%D0%90%D1%80%D0%B5%D1%84%D1%8C%D0%B5%D0%B2%D0%B0_%D0%98%D0%BB%D0%BE%D0%BD%D0%B0_%D0%98%D0%92%D0%A2_21%D0%9C%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Будем решать задачу идентификации взломщика по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в статье на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

В этом соревновании будем решать похожую задачу: алгоритм будет анализировать последовательность из нескольких веб-сайтов, посещенных подряд одним и тем же человеком, и определять, Элис это или взломщик (кто-то другой).

Данные собраны с прокси-серверов Университета Блеза Паскаля. "A Tool for Classification of Sequential Data", авторы Giacomo Kahn, Yannick Loiseau и Olivier Raynaud.

**Задача – сделать прогнозы для сессий в тестовой выборке (test_sessions.csv), определить, принадлежат ли они Элис. Сделать посылку на kaggle.**


In [ ]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#подключаю файл из папки Colab на Google диске
path_train_data = 'gdrive/My Drive/Colab_files/Catch_me/train_sessions.csv'
path_test_data = 'gdrive/My Drive/Colab_files/Catch_me/test_sessions.csv'
train_data = pd.read_csv(path_train_data)
test_data = pd.read_csv(path_test_data)

In [ ]:
print(train_data.head(n=10))
print(test_data.head(n=10))

In [ ]:
train_data.info()
test_data.info()

In [ ]:
sites_header = ['site%s' % i for i in range(1, 11)]
time_header = ['time%s' % i for i in range(1, 11)]
# Посмотрим на типы данных колонок
print(train_data.dtypes.value_counts())
# Размерность DataFrame
print('Размерность train и test:', train_data.shape, test_data.shape,)
print(train_data['site1'].value_counts(dropna = False))

# **Предобработка**

In [ ]:
# Заменим значения NA/NAN -> 0 
train_data[sites_header] = train_data[sites_header].fillna(0).astype('int')
test_data[sites_header] = test_data[sites_header].fillna(0).astype('int')

train_data[time_header] = train_data[time_header].fillna(0)
test_data[time_header] = test_data[time_header].fillna(0)
train_data[time_header]

In [ ]:
 y_train = train_data['target'].values
 X_train = train_data.drop('target', axis=1)
 print(X_train)

In [ ]:
train_data[sites_header].to_csv('preprocessing_train_set_sites.txt', sep=' ', index=None, header=None);
test_data[sites_header].to_csv('preprocessing_test_set_sites.txt', sep=' ', index=None, header=None);
from sklearn.feature_extraction.text import CountVectorizer
count_vertorizer = CountVectorizer(ngram_range=(1, 3), max_features=50000)

In [ ]:
train_sites = open('preprocessing_train_set_sites.txt')
test_sites = open('preprocessing_test_set_sites.txt')
X_train = count_vertorizer.fit_transform(train_sites)
X_test = count_vertorizer.transform(test_sites)
print(X_train.shape, X_test.shape)

# **Логистическая регрессия**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def calculate_quality_of_model(lr, X, y, split_ind):
  y_pred = lr.predict_proba(X[split_ind:, :])[:, 1]
  return roc_auc_score(y[split_ind:], y_pred)

lr = LogisticRegression(C=1.0, random_state=24, solver='saga', max_iter=500).fit(X_train, y_train)

print('100000 Score: ', calculate_quality_of_model(lr, X_train, y_train, 100000))
print('125000 Score: ', calculate_quality_of_model(lr, X_train, y_train, 125000))
print('200000 Score: ', calculate_quality_of_model(lr, X_train, y_train, 200000))

# Выделение новых признаков по времени посещения сайтов

In [ ]:
# Датафрейм для записи новых признаков времени
spent_time_data = pd.DataFrame()

times = train_data[['time%s' % i for i in range(1, 11)]]
sites = train_data[['site%s' % i for i in range(1, 11)]].fillna(0).astype(int).values

for header_index in range(1, 10):
    spent_time_data['target'] = train_data['target']
    spent_time_data['time_diff{}'.format(header_index)] = (pd.to_datetime(times['time{}'.format(header_index + 1)]) - 
                        pd.to_datetime(times['time{}'.format(header_index)])).dt.total_seconds().fillna(0)

# Сумма по строкам датафрейма - время сессии
spent_time_data['session_time'] = np.sum(spent_time_data, axis=1)
# Выделим столбец для начала работы в интернете
spent_time_data['hour'] = pd.to_datetime(times['time1']).dt.hour
# Выделим столбец для определения дня недели
spent_time_data['day_of_week'] = pd.to_datetime(times['time1']).dt.weekday
# Выделим столбец для месяца
spent_time_data['month'] = pd.to_datetime(times['time1']).dt.month
# Посчитаем количество уникальных посещенных сайтов
spent_time_data['count_unique_sites'] = [len(np.unique(session[session != 0])) for session in sites]
print(spent_time_data.head())

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6), dpi=100)
sns.countplot(x=spent_time_data['hour'], data=spent_time_data, hue='target')
plt.xlabel('Время посещения первого сайта')
plt.title('Распределение сессий по часу начала')

plt.figure(figsize=(10, 6), dpi=100)
plt.ylim(0, 1100)
sns.countplot(x=spent_time_data['hour'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во сессий')
plt.xlabel('Время посещения первого сайта')
plt.title('Сессии Элис')

Активность Элис в интернете с 12 до 13 часов и с 16 до 18 часов. Изредка появлялась в 9 и 15 часов. 

In [ ]:
plt.figure(figsize=(10, 6), dpi=100)
sns.countplot(x=spent_time_data['day_of_week'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во сессий')
plt.xlabel('День недели')
plt.grid()
plt.title('Распределение посещений по дням недели')

plt.figure(figsize=(10, 6), dpi=100)
plt.ylim(0, 1000)
sns.countplot(x=spent_time_data['day_of_week'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во сессий')
plt.xlabel('День недели')
plt.grid()
plt.title('Сессии Элис')

Элис пользуется интернетом все дни, кроме вс. Небольшая активность в ср и сб.

In [ ]:
plt.figure(figsize=(10, 6), dpi=100)
sns.countplot(x=spent_time_data['month'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во сессий')
plt.xlabel('Месяц')
plt.title('Распределение посещений по месяцам')

plt.figure(figsize=(10, 6), dpi=100)
plt.ylim(0, 500)
sns.countplot(x=spent_time_data['month'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во сессий')
plt.xlabel('Месяц')
plt.title('Сессии Элис')

Наименьшая активность Элис с мая по август и в ноябре.

In [ ]:
plt.figure(figsize=(10, 6), dpi=100)
sns.countplot(x=spent_time_data['count_unique_sites'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во посещений')
plt.xlabel('Сайт')
plt.title('График посещений уникального сайта')

plt.figure(figsize=(10, 6), dpi=100)
plt.ylim(0, 500) 
sns.countplot(x=spent_time_data['count_unique_sites'], data=spent_time_data, hue='target')
plt.ylabel('Кол-во посещений')
plt.xlabel('Сайт')
plt.title('Сессии Элис') 

Элис посещает все теже сайты что и другие люди (злоумышленник).

Таким образом, есть смысл анализировать признаки:

*   время просмотра первого сайта
*   день недели
*   месяц

In [ ]:
# Тестовая выборка с новыми признаками
test = pd.DataFrame()

time_test = test_data[['time%s' % i for i in range(1, 11)]]
sites_test = test_data[['site%s' % i for i in range(1, 11)]].fillna(0).astype(int).values

for header_index in range(1, 10):
    test['time_diff{}'.format(header_index)] = (pd.to_datetime(time_test['time{}'.format(header_index + 1)]) - 
                        pd.to_datetime(time_test['time{}'.format(header_index)])).dt.total_seconds().fillna(0)
   
test['session_time'] = np.sum(test, axis=1)
test['hour'] = pd.to_datetime(time_test['time1']).dt.hour
test['day_of_week'] = pd.to_datetime(time_test['time1']).dt.weekday
test['month'] = pd.to_datetime(time_test['time1']).dt.month
test['count_unique_sites'] = [len(np.unique(session[session != 0])) for session in sites_test]

print(test.head())

In [ ]:
# Сформируем колонки целевых признаков для новых данных
spent_time_data['target_hour'] = np.where(((spent_time_data['hour']>=12) & (spent_time_data['hour']<=13)) | ((spent_time_data['hour']>=16) & (spent_time_data['hour']<=18)), 0, 1)
test['target_hour'] = np.where(((test['hour']>=12) & (test['hour']<=13)) | ((test['hour']>=16) & (test['hour']<=18)), 1, 0)

spent_time_data['target_week_day'] = np.where((spent_time_data['day_of_week']==5) | (spent_time_data['day_of_week']==6), 1, 0)
test['target_week_day'] = np.where((test['day_of_week']==5) | (test['day_of_week']==6), 1, 0)

spent_time_data['target_month'] = np.where(((spent_time_data['month']>=5) & (spent_time_data['month']<=8)), 1, 0)
test['target_month'] = np.where(((test['month']>=5) & (test['month']<=8)), 1, 0)

target_train = spent_time_data[['target_hour', 'target_week_day','target_month']]
target_test = test[['target_hour', 'target_week_day','target_month']]

In [ ]:
Y_train_spent_time = spent_time_data['target'].values
from sklearn.model_selection import train_test_split
X_train_log, X_valid_log, Y_train_log, Y_valid_log = train_test_split(X_train, Y_train_spent_time, test_size=0.1, random_state=24)
log_reg = LogisticRegression(C=1.0, random_state=24, solver='lbfgs', max_iter=500)
log_reg.fit(X_train_log, Y_train_log)

In [ ]:
y_pred = log_reg.predict_proba(X_valid_log)
from sklearn.metrics import roc_auc_score
score_log = roc_auc_score(Y_valid_log, y_pred[:,1])
print("log",round(score_log, 2)*100)

In [ ]:
log_reg.fit(X_train, Y_train_spent_time)
Y_test = log_reg.predict_proba(X_test)
Y_test[:5]

# **Случайный лес**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

parameters = {'max_features': [3, 4, 10, 14], 'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [5, 10, 15, 20]}
rfc = RandomForestClassifier(n_estimators=50, random_state=24, 
                             n_jobs=8, oob_score=True)
gcv = GridSearchCV(rfc, parameters, n_jobs=8, cv=skf, verbose=1)
gcv.fit(X_train_log, Y_train_log)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [ ]:
gcv.best_estimator_, gcv.best_score_